# Предсказание победителя в игре Dota 2

## Загрузка и предобработка данных

Загрузим тренировочные данные в *train_data*, а тестовые в *test_data*. В дальнейшем мы объединим оба датасета в один фрейм *data* для удобства работы.

In [68]:
import pandas as pd
import numpy as np

In [69]:
train_data = pd.read_csv('./features.csv', index_col='match_id')
test_data = pd.read_csv('./features_test.csv', index_col='match_id')

Посмотрим что у интересного содержится в тренировочных данных:

In [70]:
train_data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


Признаки, которые мы будем использовать возьмем из тестовой выборки. Это значит, что мы не будем использовать признаки *duration, radiant_win, tower_status_radiant, tower_status_dire, barracks_status_radiant и barracks_status_dire*.

In [71]:
feature_columns_to_use = test_data.columns.tolist()

Проверим выборку на наличие пропусков. Сразу выберем те признаки, количество которых не равно числу строк во всей матрице,  а именно 97230:

In [72]:
train_data[feature_columns_to_use].count()[train_data[feature_columns_to_use].count() != 97230]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Видно, что некоторые признаки отсутствуют, причем в порядочном количестве. Но отсутсвие некоторых данных в этих признаков легко объяснимо, так как мы рассматриваем только первые 5 минут игры. Например *first_blood_time*, *first_blood_team*, *first_blood_player1* и *first_blood_player2* могут отсутствовать, потому что за первые 5 минут не было первой крови. В признаке *first_blood_player2* еще больше пропущенных значений, видимо потому что если один из игроков все же завалил кого-то, для второго игрока это событие могло в 5 минут и не наступить. *radiant_bottle_time*, *radiant_courier_time*, *radiant_flying_courier_time* отсутствуют, потому что игроки не успели купить соответствующие предметы, так же *radiant_first_ward_time* может отсутствовать, если никто не успел установить "наблюдателя". Для Dire команды аналогично.

Соберем тестовые и тренировочные данные, для того чтобы проводить сразу все преобразования на тестовых и на тренировочных данных:

In [73]:
data = train_data[feature_columns_to_use].append(test_data[feature_columns_to_use])

Заменил пропущенные значения в данных на нули:

In [74]:
data = data.fillna(0)

Столбец, содержащий целевую переменную, это **radiant_win**. В нем содержится 1, если победила команда Radiant и 0 в противном случае. Для начала для обучения будем использовать все столбцы матрицы признаков. В итоге мы получим следующие датафреймы для обучения, тестирования и предсказания:

In [75]:
def prepare_data(data, train_data, feature_columns):
    train_X = data[feature_columns][0:train_data.shape[0]].as_matrix()
    train_y = train_data['radiant_win']
    test_X = data[feature_columns][train_data.shape[0]::].as_matrix()
    
    return train_X, test_X, train_y

feature_columns = feature_columns_to_use
train_X, test_X, train_y = prepare_data(data, train_data, feature_columns)

## Обучение моделей. Градиентный бустинг.

Сделаем кросс-валидацию для градиентного бустинга, разобьем наши данные на 5 фолдов, предварительно перемешав. Используем различные количество количество деревьев: 10, 20, 30, 40 и 50. Посмотрим, на каком количестве деревьев будет достигнуто самое большое значение AUC-ROC.

In [76]:
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
import time, datetime

cv = KFold(train_data.shape[0], n_folds=5, shuffle=True, random_state=241)

for n_estimators in [10,20,30,40,50]:
    start_time = datetime.datetime.now()
    
    clf = GradientBoostingClassifier(n_estimators=n_estimators)
    score = cross_val_score(clf, train_X, train_y,
                            cv=cv, scoring='roc_auc').mean()
    
    print('{0} estimators score = {1:.3f}'.format(n_estimators, score))
    print('Time elapsed: {0}'.format(datetime.datetime.now() - start_time))

10 estimators score = 0.664
Time elapsed: 0:00:36.511622
20 estimators score = 0.683
Time elapsed: 0:01:08.812752
30 estimators score = 0.689
Time elapsed: 0:01:40.918995
40 estimators score = 0.694
Time elapsed: 0:02:13.665368
50 estimators score = 0.697
Time elapsed: 0:02:45.839476


Мы получили лучший результат на классификаторе из 50 деревьев. В целом все работает быстро, поэтому имеет смысл продолжить увеличение количества деревьев. Для 30 деревьев кросс-валидация проводилась за 1 минуту и 41 секунду, при этом качество предсказаний равно 0.689. Очевидно 30 деревьев не предел и имеет смысл использовать больше.

Для увеличения скорости обучения модели, можно исключить признаки, делающие наименьший вклад в предсказание модели, а так же поиграться с максимальной глубиной деревьев (параметр *max_depth*) и скоростью обучения (параметр *learning_rate*). Уменьшая глубину деревьев (по-умолчанию 3) и увеличивая скорость (по-умолчанию 0.1) можно уменьшить время обучения модели, но так же возможно ухудшить качество обучения.

По-пробуем увеличить глубину деревьев и уменьшить learning_rate для увеличения результата

In [37]:
n_estimators = 50
max_depth=6

for learning_rate in [0.1, 0.05]:
    start_time = datetime.datetime.now()

    clf = GradientBoostingClassifier(n_estimators=n_estimators,
                                     max_depth=max_depth,
                                     learning_rate=learning_rate
                                    )
    score = cross_val_score(clf, train_X, train_y,
                            cv=cv, scoring='roc_auc').mean()

    print('lr={0} estimators score = {1:.3f}'.format(learning_rate, score))
    print('Time elapsed: {0}'.format(datetime.datetime.now() - start_time))

lr=0.1 estimators score = 0.706
Time elapsed: 0:08:29.495611
lr=0.05 estimators score = 0.701
Time elapsed: 0:08:32.654476


Отлично, мы немного улучшили наши показатели. Однако и вреся выросло существенно: с 2 минут 45 секунд до 8 минут 29 секунд. При этом результат с learning_rate равным 0.1 оказался даже лучше, чем с 0.05. Остановимся на параметрах n_estimators=50, learning_rate=0.1, и max_depth=6.

## Логистическая регрессия.

Построим теперь модель логистической регрессии. Нормируем признаки с помощью StandartScaler. Будем использовать L2 регулязацию и будем подбирать параметр регулизации C в интервале от $10^{-5}$ до $10^6$:

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

def scale_data(data, train_X, test_X):
    ss = StandardScaler()
    ss.fit(data)
    train_X_scaled = ss.transform(train_X)
    test_X_scaled = ss.transform(test_X)    
    return train_X_scaled, test_X_scaled
    
train_X_scaled, test_X_scaled = scale_data(data, train_X, test_X)

In [78]:
for C in np.power(10.0, np.arange(-5,6)):
    start_time = datetime.datetime.now()

    clf = LogisticRegression(C=C,  penalty='l2', n_jobs=-1)
    score = cross_val_score(clf, train_X_scaled, train_y,
                            cv=cv, scoring='roc_auc').mean()

    print('C={0} estimators score = {1:.7f}'.format(C, score))
    print('Time elapsed: {0}'.format(datetime.datetime.now() - start_time))

C=1e-05 estimators score = 0.6951351
Time elapsed: 0:00:03.634230
C=0.0001 estimators score = 0.7112499
Time elapsed: 0:00:05.692641
C=0.001 estimators score = 0.7161798
Time elapsed: 0:00:10.972737
C=0.01 estimators score = 0.7163415
Time elapsed: 0:00:14.474826
C=0.1 estimators score = 0.7163100
Time elapsed: 0:00:15.139032
C=1.0 estimators score = 0.7163066
Time elapsed: 0:00:15.031594
C=10.0 estimators score = 0.7163064
Time elapsed: 0:00:15.053311
C=100.0 estimators score = 0.7163062
Time elapsed: 0:00:15.066369
C=1000.0 estimators score = 0.7163062
Time elapsed: 0:00:15.052998
C=10000.0 estimators score = 0.7163062
Time elapsed: 0:00:15.074319
C=100000.0 estimators score = 0.7163062
Time elapsed: 0:00:15.080239


Мы видим, что в данном случае логистическая регрессия превзошла результаты градиентного бустинга. Для параметра *C=0.01* мы получили значение AUC ROC равное 0.7163415. Логистическая регрессия работает намного быстрее бустинга: всего примерно за ~14 секунд, но при этом не проигрывает в качестве. Возможно результат зависит линейно от некоторых параметров, типа количества золота, опыта и числа убитых противников, а логистическая регрессия хорошо смогла использовать эти параметры, построив разделяющую гипперплоскость.

Удалим из данных категориальные признаки, оставив только числовые:

In [91]:
categorial_features = ['lobby_type', 'r1_hero', 'r2_hero',
                       'r3_hero', 'r4_hero', 'r5_hero',
                       'd1_hero', 'd2_hero', 'd3_hero',
                       'd4_hero', 'd5_hero' ]

new_feature_columns = [x for x in feature_columns_to_use if x not in categorial_features]
train_X, test_X, train_y = prepare_data(data, train_data, new_feature_columns)
train_X_scaled, test_X_scaled = scale_data(data[new_feature_columns], train_X, test_X)

Попробуем снова:

In [80]:
for C in np.power(10.0, np.arange(-5,6)):
    start_time = datetime.datetime.now()

    clf = LogisticRegression(C=C,  penalty='l2', n_jobs=-1)
    score = cross_val_score(clf, train_X_scaled, train_y,
                            cv=cv, scoring='roc_auc').mean()

    print('C={0} estimators score = {1:.7f}'.format(C, score))
    print('Time elapsed: {0}'.format(datetime.datetime.now() - start_time))

C=1e-05 estimators score = 0.6950712
Time elapsed: 0:00:03.405743
C=0.0001 estimators score = 0.7112484
Time elapsed: 0:00:05.313801
C=0.001 estimators score = 0.7162359
Time elapsed: 0:00:10.149254
C=0.01 estimators score = 0.7164009
Time elapsed: 0:00:13.158288
C=0.1 estimators score = 0.7163739
Time elapsed: 0:00:13.643077
C=1.0 estimators score = 0.7163707
Time elapsed: 0:00:13.819297
C=10.0 estimators score = 0.7163705
Time elapsed: 0:00:13.816106
C=100.0 estimators score = 0.7163706
Time elapsed: 0:00:13.803889
C=1000.0 estimators score = 0.7163706
Time elapsed: 0:00:13.794795
C=10000.0 estimators score = 0.7163706
Time elapsed: 0:00:13.836257
C=100000.0 estimators score = 0.7163706
Time elapsed: 0:00:13.831195


Мы видим, что результат совсем немного улучшился, при том же параметра *C=0.01* результат равен 0.7164009. Т.е. категориальные признаки отрицательно влияли на процесс обучения и просто выкинув их мы смогли улучшить результат. Логистическая регрессия пытается интерпритировать категориальные признаки как числовые характеристики и строить на основании них зависимость, что на самом деле не имеет физического смысла.

Попробуем использовать мешок слов и заменить категориальные признаки о герои игроков на дамми признаки: сделаем N дополнительных признаков по количеству возможных героев и при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [135]:
heroes_features = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                   'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero' ]

unique_values = set()
for feature in heroes_features:
    unique_values |= set(np.unique(data[feature]))
    
len(unique_values)

108

In [137]:
np.unique(data['r1_hero'])

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112])

Видим что у нас всего 108 возможных видов героев, пронумерованных от 1 до 112.

In [83]:
N = 112
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

heroes_features = ['hero_N_{}'.format(i) for i in range(1,113)]
heroes = pd.DataFrame(X_pick, columns=heroes_features, index=data.index)

Проверим логистическую регрессию на новых данных. Не будем скейлить вновь добавленные фичи героев для начала:

In [99]:
train_X_scaled_with_heroes = np.hstack((train_X_scaled,
                                        X_pick[0:train_data.shape[0]]))
test_X_scaled_with_heroes = np.hstack((test_X_scaled,
                                        X_pick[train_data.shape[0]::]))

In [100]:
for C in np.power(10.0, np.arange(-5,2)):
    start_time = datetime.datetime.now()

    clf = LogisticRegression(C=C,  penalty='l2', n_jobs=-1)
    score = cross_val_score(clf, train_X_scaled_with_heroes, train_y,
                            cv=cv, scoring='roc_auc').mean()

    print('C={0} estimators score = {1:.7f}'.format(C, score))
    print('Time elapsed: {0}'.format(datetime.datetime.now() - start_time))

C=1e-05 estimators score = 0.6991850
Time elapsed: 0:00:03.539834
C=0.0001 estimators score = 0.7250197
Time elapsed: 0:00:05.850475
C=0.001 estimators score = 0.7462956
Time elapsed: 0:00:12.286569
C=0.01 estimators score = 0.7517357
Time elapsed: 0:00:20.461273
C=0.1 estimators score = 0.7519375
Time elapsed: 0:00:27.972637
C=1.0 estimators score = 0.7519197
Time elapsed: 0:00:29.336793
C=10.0 estimators score = 0.7519168
Time elapsed: 0:00:30.054246


Качество предсказаний на кросс-валидации существенно выросло. Сейчас максисмум равен 0.7519375 при параметре *C=0.1*. Видно, что признаки - герои, которые учавствовали в битве существенно влияет на качество предсказания. После того как мы привели данные признаки к виду, с которыми нормально может работать логистическая регрессия, качество модели значительно улучшилось. Теперь отсутствует проблема неправильной интерпретации этих признаков.

А теперь тоже самое, но скейлим все данные:

In [102]:
data_with_heroes = pd.concat([data, heroes], axis=1)

train_X, test_X, train_y = prepare_data(
    data_with_heroes[new_feature_columns+heroes_features],
    train_data,
    new_feature_columns+heroes_features)

train_X_scaled_with_heroes, test_X_scaled_with_heroes = scale_data(
                    data_with_heroes[new_feature_columns+heroes_features],
                    train_X, test_X)

In [103]:
for C in np.power(10.0, np.arange(-5,2)):
    start_time = datetime.datetime.now()

    clf = LogisticRegression(C=C,  penalty='l2', n_jobs=-1)
    score = cross_val_score(clf, train_X_scaled_with_heroes, train_y,
                            cv=cv, scoring='roc_auc').mean()

    print('C={0} estimators score = {1:.7f}'.format(C, score))
    print('Time elapsed: {0}'.format(datetime.datetime.now() - start_time))

C=1e-05 estimators score = 0.7148443
Time elapsed: 0:00:06.135694
C=0.0001 estimators score = 0.7428481
Time elapsed: 0:00:09.754201
C=0.001 estimators score = 0.7516695
Time elapsed: 0:00:18.618962
C=0.01 estimators score = 0.7519703
Time elapsed: 0:00:25.167908
C=0.1 estimators score = 0.7519247
Time elapsed: 0:00:27.098209
C=1.0 estimators score = 0.7519175
Time elapsed: 0:00:27.787238
C=10.0 estimators score = 0.7519169
Time elapsed: 0:00:27.758886


Качество немного улучшилось, получили 0.7519703 при значении параметра *C=0.01*. 

Это лучшая модель из всех описанных, с помощью нее будут строиться итоговые предсказания.

## Строим предсказания для тестовых данных.

In [109]:
clf = LogisticRegression(C=0.01,  penalty='l2', n_jobs=-1)
clf.fit(train_X_scaled_with_heroes, train_y)
pred_y = clf.predict_proba(test_X_scaled_with_heroes)[:, 1]

In [110]:
predicts = pd.DataFrame(pred_y, index=data[train_X.shape[0]::].index,
                        columns=['radiant_win'])

In [111]:
predicts.head()

,radiant_win
match_id,
6,0.822707
7,0.752106
10,0.188938
13,0.857577
16,0.243813


Найдем минимальное и максимальное значение:

In [112]:
predicts.describe()

,radiant_win
count,17177.000000
mean,0.516926
std,0.220227
min,0.008491
25%,0.346838
50%,0.521891
75%,0.690636
max,0.996278


Минимальное значение равно 0.008491, максимальное 0.996278, в среднем модель предсказала победу Radiant в приблизительно 52% случаев.

Сохраним предсказанные значения в файл:

In [113]:
predicts.to_csv('predicts.csv.gz', compression='gzip', float_format='%.6f')

![](http://i.imgur.com/3cxtkW8.jpg)